In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load dataset (assuming a CSV file with features and target price column)
data = pd.read_csv("/content/drive/MyDrive/BTECH/2nd Year/4th Sem/LAB/Generative AI/Housing.csv")

# Splitting features and target
X = data.drop(columns=["price"])  # Assuming 'price' is the target column
y = data["price"]

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features (e.g., columns with 'object' dtype)
categorical_features = X_train.select_dtypes(include=['object']).columns

# Create a ColumnTransformer to handle numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude=['object']).columns),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),  # Use OneHotEncoder for categorical features
    ])

# Apply preprocessing to training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Define the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")

# Save the model
model.save("housing_price_model.h5")

# Load and use the model for prediction
def predict_price(input_features):
    input_features = pd.DataFrame([input_features], columns=X.columns)  # Convert input to DataFrame
    input_features = preprocessor.transform(input_features)  # Apply preprocessing
    prediction = model.predict(input_features)
    return prediction[0][0]

# Example usage
sample_input = X_test[0]  # Assuming X_test is a NumPy array after transformation
predicted_price = predict_price(X.iloc[0])  # Use original data for sample input
print(f"Predicted Price: {predicted_price}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import make_regression, make_classification

# Choose the problem type: 'regression' or 'classification'
problem_type = 'regression'  # Change to 'classification' for classification problems

if problem_type == 'regression':
    # Generate synthetic regression data
    X, y = make_regression(n_samples=100, n_features=1, noise=10, random_state=42)
    model = LinearRegression()
else:
    # Generate synthetic classification data
    X, y = make_classification(n_samples=100, n_features=5, random_state=42)
    model = LogisticRegression()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

if problem_type == 'regression':
    # Compute regression metrics
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_rmse = train_mse ** 0.5
    test_rmse = test_mse ** 0.5
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    print(f"Training MAE: {train_mae}, Testing MAE: {test_mae}")
    print(f"Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")
    print(f"Training R²: {train_r2}, Testing R²: {test_r2}")
else:
    # Compute classification metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"Training Accuracy: {train_acc}, Testing Accuracy: {test_acc}")
    print("Classification Report:\n", classification_report(y_test, y_test_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "Housing.csv"
df = pd.read_csv("/content/drive/MyDrive/BTECH/2nd Year/4th Sem/LAB/Generative AI/Housing.csv")

# Identify categorical and numerical columns
categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating',
                       'airconditioning', 'prefarea', 'furnishingstatus']
numerical_columns = [col for col in df.columns if col not in categorical_columns + ['price']]

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Handle skewness in target variable
df['price'] = np.log1p(df['price'])

# Separate features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Normalize numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the improved ANN model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32,
          validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

# Evaluate the model
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Training MAE: {train_mae}, Testing MAE: {test_mae}')

# Save the model
model.save("/mnt/data/housing_price_model_optimized.h5")
print("Optimized model saved as 'housing_price_model_optimized.h5'.")

# Load and use the saved model
loaded_model = tf.keras.models.load_model("/mnt/data/housing_price_model_optimized.h5")
predictions = np.expm1(loaded_model.predict(X_test[:5]))

# Display sample predictions
print("Sample Predictions:")
print(predictions)